In [10]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pylab
from pylab import rcParams
rcParams['figure.figsize'] = 10, 7
#import astroML 
#from astroML.plotting import setup_text_plots
#setup_text_plots(fontsize=14, usetex=True)

Using matplotlib backend: Qt5Agg


In [3]:
# this functions may be for johnson instead of stromgren
def Teff(bv):
    return 7540./(bv+0.865)
def L(V,d):
    return (V-5*np.log10(d/10)-4.83)/2.5

In [4]:
stars = np.loadtxt('isochrone/stars_std.lst')
stars_Teff_L = np.empty((np.shape(stars)[0],2)) # Teff and luminosity from V and b-v 
#extrapolated using a formula in Mariona's notes. It contains Teff,L 
# in the second dimension
stars_Teff_L[:,0] = Teff(stars[:,2])
d=830
stars_Teff_L[:,1] = -L(stars[:,0],d)

iso1 = np.loadtxt('isochrone/iso_4p2Gyr_z0_d1Kpc_r0p03.dat')
iso2 = np.loadtxt('isochrone/iso_4p2Gyr_z0_d830pc_r0p03.dat')
iso3 = np.loadtxt('isochrone/iso_4p4Gyr_z0_d830pc_r0p03.dat')
iso4 = np.loadtxt('isochrone/iso_a4000Myr_z0p01_d850pc_r0p04.dat')
iso5 = np.loadtxt('isochrone/iso_a5000Myr_z0p02_d1Mpc_r0')

In [5]:
plt.plot(stars[:,2], stars[:,0],'.') #x-axis: b-y; y-axis: V
plt.plot(iso1[:,1], iso1[:,0], '-', label='iso1')
plt.plot(iso2[:,1], iso2[:,0], '-', label='iso2')
plt.plot(iso3[:,1], iso3[:,0], '-', label='iso3')
#plt.plot(10**iso4[:,3], iso4[:,2], '-', label='iso4')
#plt.plot(10**iso5[:,3], iso5[:,2], '-', label='iso5')
plt.ylim([20, 9])
#plt.xlim([10000, 3000])
plt.xlabel('b-y')
plt.ylabel('V')
plt.legend()

In [6]:
# la interpolació s'ha de fer només per la main sequence, 
# per tant cal tallar al màxim per eliminar la relacio bv Teff 
# post MS, per gegants i en proces de se-ho, la funció és bastant 
# maca
from scipy.signal import argrelextrema
#sorted_index = np.argsort(iso5[:Tmax,0])
#sorted_by = iso5[sorted_index,1]
maxInd = argrelextrema(iso5[:,3], np.less)[0][0]
by = iso5[:maxInd,1]
Teff = iso5[:maxInd,3]
# els vaors de la isocrona amb index menor a maxInd son aquells 
# que pertanyen a la main sequence, per tant al fer el fit/interpolacio
# només hem de tenir en compte els vectors de 0:maxInd
plt.figure()
plt.plot(by,Teff,'.')
plt.xlabel('b-y')
plt.ylabel('Teff')
plt.figure()
plt.plot(iso5[:maxInd,0],iso5[:maxInd,2],'.') 
plt.xlabel('V')
plt.ylabel('$L/L_\odot$')

Text(0,0.5,u'$L/L_\\odot$')

In [7]:
# fem el fit en questió. aixo no ho he borrat perque va be per 
# veure com es comporten les funcions fora de la zona MS.
# així, com més lineal és menys modifica sense sentit els resultats
# de les estrelles fora de la MS.
def curvefitter(x,p0,plow,phigh,pchange,prate):
    # params has dimension 5 (from 0 to 4)
    #y = p0+p1*x+p2*x**2+p3*x**3+p_1/x
    # try with variable p1
    p1 = (phigh-plow)/(1+np.exp(prate*(x-pchange)))+plow
    y = p0+p1*(x-pchange)
    return y
#p1_0 = (Teff[-1]-Teff[0])/(by[-1]-by[0]) #first guess of params[1]
plow = (Teff[10]-Teff[0])/(by[10]-by[0])
phigh = (Teff[-1]-Teff[-10])/(by[-1]-by[-10])
p0_0 = Teff[0]-plow*by[0]
p0 = [p0_0,plow,phigh,.6,50]
print p0
import scipy.optimize as opt
popt, pcov = opt.curve_fit(curvefitter,by,Teff,p0)
plt.figure()
plt.plot(by,Teff,'+')
by_grid = np.linspace(.3*min(by),1.7*max(by),1000)
plt.plot(by_grid,curvefitter(by_grid,*popt),'-')
print popt
plt.xlabel('b-y')
plt.ylabel('Teff')

[3.8010285714285668, -0.23809523809523281, -0.49999999999972244, 0.6, 50]
[  3.66654893  -0.27343116  -0.46888241   0.60768322  14.38758623]


Text(0,0.5,u'Teff')

In [8]:
def Vby2LT(stars,iso):
    # 1- get indMax
    maxInd = argrelextrema(iso[:,3], np.less)[0][0]
    by = iso[:maxInd,1]
    T  = iso[:maxInd,3]
    V  = iso[:maxInd,0]
    L  = iso[:maxInd,2]
    # 2- fit the curves
    # 2.1- fit T(by)
    plow = (T[10]-T[0])/(by[10]-by[0])
    phigh = (T[-1]-T[-10])/(by[-1]-by[-10])
    p0_0 = T[0]-plow*by[0]
    p0 = [p0_0,plow,phigh,.6,50]
    poptT, pcov = opt.curve_fit(curvefitter,by,T,p0)
    # 2.1- fit T(by)
    plow = (L[10]-L[0])/(V[10]-V[0])
    phigh = (L[-1]-L[-10])/(V[-1]-V[-10])
    p0_0 = L[0]-plow*V[0]
    p0 = [p0_0,plow,phigh,.6,50]
    poptL, pcov = opt.curve_fit(curvefitter,V,L,p0)
    # 3- apply the curve with the fitted parameters to V and by to 
    # get L and T
    Lstars = curvefitter(stars[:,0],*poptL)
    Tstars = curvefitter(stars[:,2],*poptT)
    return Tstars,Lstars

In [41]:
Tstars,Lstars = Vby2LT(stars,iso2)
plt.figure()
plt.semilogy(10**iso2[:,3],10**iso2[:,2],'-',label='Best isochrone fit',color='purple')
plt.scatter(10**Tstars,10**Lstars,c=10**Tstars,label='Observations',cmap=cm.RdYlBu, vmin=3000,
            vmax=9000,marker='.',s=0.8)#,linewidths=0.1,edgecolors='k')
plt.xlim([10000, 3000])
plt.xlabel('$T_{eff}$')
plt.ylabel('$L/L_\odot$')
plt.gca().set_facecolor('xkcd:almost black') # sensible options: charcoal, dark, almost black and midnight. Black is too much
#plt.grid(True,color='xkcd:light grey', linestyle='--', linewidth=1, alpha=0.6)
plt.legend()

C:\Users\Oriol\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


## Generate the list of pst stars

In [65]:
pst_b = np.loadtxt('Lab1/science_m67_b.pst.1',dtype=np.float64)
pst_y = np.loadtxt('Lab1/science_m67_y.pst.1',dtype=np.float64)

In [70]:
print 'b filter\n'
for i,(num,x,y,m,ms) in enumerate(pst_b):
    print '%d & %.3f & %.3f & %.3f & %.7g \\\\' %(num,x,y,m,ms)
print '\ny filter\n'
for i,(num,x,y,m,ms) in enumerate(pst_y):
    print '%d & %.3f & %.3f & %.3f & %.7g \\\\' %(num,x,y,m,ms)

b filter

175 & 1945.945 & 876.506 & -11.745 & 1.119405 \\
24 & 1829.076 & 114.152 & -11.659 & -0.9860825 \\
235 & 213.076 & 1145.705 & -11.611 & -1.427757 \\
375 & 1740.711 & 1895.507 & -11.581 & -1.485688 \\
226 & 358.469 & 1101.716 & -11.399 & -3.561035 \\
354 & 1176.973 & 1711.751 & -11.259 & -3.470577 \\
36 & 570.607 & 180.730 & -11.212 & -5.184465 \\
203 & 866.194 & 1006.945 & -11.080 & -4.050434 \\
222 & 1540.214 & 1097.801 & -11.020 & -3.684947 \\
12 & 1225.507 & 52.826 & -10.981 & -6.149918 \\
35 & 798.813 & 179.623 & -10.855 & -6.703156 \\
109 & 705.694 & 616.636 & -10.707 & -5.814357 \\
428 & 444.344 & 2255.923 & -10.705 & -5.576495 \\
241 & 1377.805 & 1173.293 & -10.654 & -6.27722 \\
66 & 812.461 & 318.594 & -10.494 & -7.911538 \\
227 & 1580.952 & 1112.471 & -10.454 & -5.203313 \\
309 & 725.354 & 1459.743 & -10.444 & -6.486355 \\
65 & 504.365 & 314.500 & -10.385 & -8.76615 \\
31 & 1436.586 & 157.905 & -10.313 & -9.385581 \\
86 & 176.649 & 478.417 & -10.247 & -9.382069 \\
28